In [ ]:
import zipfile
with zipfile.ZipFile('KAGGLE_dataset_fashion.zip', 'r') as zip_ref:
  zip_ref.extractall('dataset')



In [ ]:
import os
print(os.listdir('dataset'))

['myntradataset', 'images', 'styles.csv']


In [ ]:
import pandas as pd
df = pd.read_csv('dataset/styles.csv', on_bad_lines='skip')
print(df.sample(5))

          id gender masterCategory subCategory      articleType baseColour  \
16458   7430  Women        Apparel     Topwear           Tunics     Purple   
6187   18548  Women       Footwear       Shoes     Casual Shoes      Beige   
25969  19385    Men        Apparel   Innerwear  Innerwear Vests       Grey   
13239  52585  Women    Accessories        Bags         Clutches      Green   
38324  56983  Women       Footwear       Shoes            Flats       Blue   

       season    year   usage                             productDisplayName  
16458  Summer  2011.0  Casual  W Women Square Neck Short Sleeve Purple Tunic  
6187   Winter  2015.0  Casual              Catwalk Women  Beige Casual Shoes  
25969  Summer  2016.0  Casual     Crusoe Men Grey Xtreme Game Innerwear Vest  
13239  Summer  2012.0  Casual               Mod'acc Women Light Green Clutch  
38324  Winter  2012.0  Casual                          HM Women Blue Sandals  


In [6]:
from PIL import Image
img_path = 'dataset/images/10001.jpg'
Image.open(img_path).show()

In [14]:
df['text'] = df['productDisplayName'].fillna('')
df['text'] = df['text'].str.lower()


In [16]:
few_shot_df = df.groupby('masterCategory')['id'].apply(lambda x: x.sample(min(len(x), 10))) \
.reset_index().merge(df, on=['masterCategory', 'id'])

In [17]:
!pip install ftfy regex tqdm
!pip install git+https://github.com/openai/CLIP.git
!pip install peft  # for LoRA adapters[web:43][web:47]


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 2.2 MB/s eta 0:00:00
  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-o3lw7emb
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /tmp/pip-req-build-o3lw7emb
  Resolved https://github.com/openai/CLIP.git to commit dcba3cb2e2827b402d2701e7e1c7d9fed8a20ef1
  Preparing metadata (setup.py) ... done
  Created wheel for clip: filename=clip-1.0-py3-none-any.whl size=1369490 sha256=0e7b8b838f4eb81effd592050425d708f3fc133bdad344361f91779b0d027123
  Stored in directory: /tmp/pip-ephem-wheel-cache-zvsbklan/wheels/35/3e/df/3d24cbfb3b6a06f17a2bfd7d1138900d4365d9028aa8f6e92f
Successfully built clip


In [18]:
import clip
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load("ViT-B/32", device=device)



100%|███████████████████████████████████████| 338M/338M [00:04<00:00, 85.2MiB/s]


In [21]:

image = preprocess(Image.open(img_path)).unsqueeze(0).to(device)
image_emb = model.encode_image(image)

text = clip.tokenize(["sample product text from your df"]).to(device)
text_emb = model.encode_text(text)


In [22]:
multimodal_emb = torch.cat([image_emb, text_emb], dim=1)



In [25]:
train_df = few_shot_df.sample(frac=0.8)
val_df = few_shot_df.drop(train_df.index)


In [26]:
import torch.nn as nn

class MultimodalClassifier(nn.Module):
    def __init__(self, emb_dim, num_classes):
        super().__init__()
        self.fc = nn.Linear(emb_dim, num_classes)
    def forward(self, x):
        return self.fc(x)




In [27]:
!pip install gradio


In [28]:
import gradio as gr

def predict_fn(img, text):
    img_emb = model.encode_image(preprocess(img).unsqueeze(0).to(device))
    text_emb = model.encode_text(clip.tokenize([text]).to(device))
    concat_emb = torch.cat([img_emb, text_emb], dim=1)
    pred = classifier(concat_emb)
    category = categories[torch.argmax(pred)]
    return category

gr.Interface(fn=predict_fn, inputs=["image", "text"], outputs="label").launch(share=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://af99ff671b80ee59c1.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [29]:
from PIL import Image
import numpy as np

def preprocess_image(img):
    if isinstance(img, np.ndarray):
        img = Image.fromarray(img)
    return preprocess(img).unsqueeze(0).to(device)


In [30]:
model.eval()
classifier.eval()


NameError: name 'classifier' is not defined

In [33]:
import torch.nn as nn

# Example: If CLIP image & text embeddings size is 512 each (ViT-B/32 model)
embedding_dim = 512 * 2  # concatenating image and text embeddings
category_list = few_shot_df['masterCategory'].unique().tolist()
num_classes = len(category_list)  # number of categories in your dataset

class MultimodalClassifier(nn.Module):
    def __init__(self, emb_dim, n_classes):
        super().__init__()
        self.fc = nn.Linear(emb_dim, n_classes)
    def forward(self, x):
        return self.fc(x)

# Instantiate classifier
classifier = MultimodalClassifier(embedding_dim, num_classes).to(device)

# If you have saved weights after training, load them:
# classifier.load_state_dict(torch.load('path_to_saved_weights.pth'))

## Summary:

### Data Analysis Key Findings

* The `FashionDataset` class was successfully created to handle the loading and preprocessing of image and text data using CLIP.
* PyTorch `DataLoader` objects were successfully created for training and validation datasets with a batch size of 32, with the training data shuffled.
* The `CrossEntropyLoss` function was chosen as the criterion for classification, and the Adam optimizer was selected with a learning rate of 0.001 to update the classifier's parameters.
* The training process ran for 10 epochs, showing a consistent decrease in training loss.
* Validation loss decreased initially but then increased, while validation accuracy plateaued at 16.67% after the third epoch.
* The final evaluation on the validation set resulted in a loss of 1.9427 and an accuracy of 16.67%.

### Insights or Next Steps

* The low validation accuracy suggests that the classifier is not performing well, likely due to overfitting to the small training dataset.
* Next steps could include exploring data augmentation techniques, using a pre-trained classifier, or increasing the size of the training dataset to improve generalization.

## Evaluate the trained classifier

### Subtask:
Evaluate the final performance of the trained classifier on the validation set.

**Reasoning**:
Evaluate the final performance of the trained classifier on the validation set by calculating the total validation loss and accuracy.

In [43]:
model.eval()
classifier.eval()

correct = 0
total = 0
val_loss = 0.0

with torch.no_grad():
    for images, texts, labels in val_dataloader:
        images = images.to(device)
        texts = texts.to(device)
        labels = labels.to(device)

        image_embeddings = model.encode_image(images)
        text_embeddings = model.encode_text(texts)

        concatenated_embeddings = torch.cat([image_embeddings, text_embeddings], dim=1)
        outputs = classifier(concatenated_embeddings)
        loss = criterion(outputs, labels)

        val_loss += loss.item() * images.size(0)

        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

epoch_val_loss = val_loss / len(val_dataset)
accuracy = 100 * correct / total

print(f'Final Validation Loss: {epoch_val_loss:.4f}, Final Validation Accuracy: {accuracy:.2f}%')

Final Validation Loss: 2.3002, Final Validation Accuracy: 16.67%


## Train the classifier

### Subtask:
Write the training loop to iterate over the dataloaders, perform forward and backward passes, and update the model weights. Include evaluation on the validation set during training to monitor performance.

**Reasoning**:
Implement the training loop including forward and backward passes, optimizer steps, and evaluation on the validation set, following steps 1-15 of the instructions.

In [42]:
num_epochs = 10

for epoch in range(num_epochs):
    # Training
    model.train()
    classifier.train()
    running_loss = 0.0
    for images, texts, labels in train_dataloader:
        images = images.to(device)
        texts = texts.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()

        with torch.no_grad():
            image_embeddings = model.encode_image(images)
            text_embeddings = model.encode_text(texts)

        concatenated_embeddings = torch.cat([image_embeddings, text_embeddings], dim=1)
        outputs = classifier(concatenated_embeddings)
        loss = criterion(outputs, labels)

        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)

    epoch_loss = running_loss / len(train_dataset)
    print(f'Epoch {epoch+1}/{num_epochs}, Training Loss: {epoch_loss:.4f}')

    # Validation
    model.eval()
    classifier.eval()
    correct = 0
    total = 0
    val_loss = 0.0
    with torch.no_grad():
        for images, texts, labels in val_dataloader:
            images = images.to(device)
            texts = texts.to(device)
            labels = labels.to(device)

            image_embeddings = model.encode_image(images)
            text_embeddings = model.encode_text(texts)

            concatenated_embeddings = torch.cat([image_embeddings, text_embeddings], dim=1)
            outputs = classifier(concatenated_embeddings)
            loss = criterion(outputs, labels)

            val_loss += loss.item() * images.size(0)

            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    epoch_val_loss = val_loss / len(val_dataset)
    accuracy = 100 * correct / total
    print(f'Epoch {epoch+1}/{num_epochs}, Validation Loss: {epoch_val_loss:.4f}, Validation Accuracy: {accuracy:.2f}%')

Epoch 1/10, Training Loss: 0.7343
Epoch 1/10, Validation Loss: 1.9730, Validation Accuracy: 16.67%
Epoch 2/10, Training Loss: 0.6407
Epoch 2/10, Validation Loss: 1.9819, Validation Accuracy: 16.67%
Epoch 3/10, Training Loss: 0.5704
Epoch 3/10, Validation Loss: 2.0013, Validation Accuracy: 16.67%
Epoch 4/10, Training Loss: 0.5094
Epoch 4/10, Validation Loss: 2.0400, Validation Accuracy: 16.67%
Epoch 5/10, Training Loss: 0.4546
Epoch 5/10, Validation Loss: 2.0795, Validation Accuracy: 16.67%
Epoch 6/10, Training Loss: 0.4064
Epoch 6/10, Validation Loss: 2.1256, Validation Accuracy: 16.67%
Epoch 7/10, Training Loss: 0.3647
Epoch 7/10, Validation Loss: 2.1739, Validation Accuracy: 16.67%
Epoch 8/10, Training Loss: 0.3267
Epoch 8/10, Validation Loss: 2.2200, Validation Accuracy: 16.67%
Epoch 9/10, Training Loss: 0.2949
Epoch 9/10, Validation Loss: 2.2603, Validation Accuracy: 16.67%
Epoch 10/10, Training Loss: 0.2675
Epoch 10/10, Validation Loss: 2.3002, Validation Accuracy: 16.67%


## Define loss function and optimizer

### Subtask:
Choose an appropriate loss function (e.g., CrossEntropyLoss for classification) and an optimizer (e.g., Adam).

**Reasoning**:
Instantiate CrossEntropyLoss and the Adam optimizer.

In [41]:
import torch.nn as nn
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(classifier.parameters(), lr=0.001)

## Create data loaders

### Subtask:
Create PyTorch DataLoaders for the training and validation datasets to efficiently batch and load the data during training.

**Reasoning**:
Instantiate the FashionDataset for train and validation dataframes and create DataLoaders for them.

In [40]:
from torch.utils.data import DataLoader

train_dataset = FashionDataset(train_df, 'dataset/images', model, preprocess)
val_dataset = FashionDataset(val_df, 'dataset/images', model, preprocess)

batch_size = 32
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

## Create a custom dataset

### Subtask:
Define a PyTorch Dataset to handle loading images and text data from the `few_shot_df`, preprocess the images and tokenize the text, and return the image embeddings, text embeddings, and corresponding labels.

**Reasoning**:
I need to define a PyTorch Dataset class to handle the data loading and preprocessing for training the multimodal classifier. This involves creating a custom class that inherits from `torch.utils.data.Dataset` and implementing the `__init__`, `__len__`, and `__getitem__` methods as described in the instructions. I will also import the necessary libraries.

In [39]:
from torch.utils.data import Dataset
import os
from PIL import Image
import clip
import torch

class FashionDataset(Dataset):
    def __init__(self, dataframe, img_dir, clip_model, preprocess_fn):
        self.dataframe = dataframe
        self.img_dir = img_dir
        self.clip_model = clip_model
        self.preprocess_fn = preprocess_fn
        self.categories = dataframe['masterCategory'].unique().tolist()
        self.category_to_label = {category: i for i, category in enumerate(self.categories)}

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        img_id = self.dataframe.iloc[idx]['id']
        text = self.dataframe.iloc[idx]['productDisplayName']
        category = self.dataframe.iloc[idx]['masterCategory']

        img_path = os.path.join(self.img_dir, str(img_id) + '.jpg')
        image = Image.open(img_path)
        image_tensor = self.preprocess_fn(image)

        text_tokens = clip.tokenize([text])[0]

        label = self.category_to_label[category]

        return image_tensor, text_tokens, label

# Task
Train a classifier using the `few_shot_df` dataframe.

## Create a custom dataset

### Subtask:
Define a PyTorch Dataset to handle loading images and text data from the `few_shot_df`, preprocess the images and tokenize the text, and return the image embeddings, text embeddings, and corresponding labels.


**Reasoning**:
I need to define a PyTorch Dataset class to handle the data loading and preprocessing for training the multimodal classifier. This involves creating a custom class that inherits from `torch.utils.data.Dataset` and implementing the `__init__`, `__len__`, and `__getitem__` methods as described in the instructions. I will also import the necessary libraries.



In [34]:
from torch.utils.data import Dataset
import os
from PIL import Image
import clip
import torch

class FashionDataset(Dataset):
    def __init__(self, dataframe, img_dir, clip_model, preprocess_fn):
        self.dataframe = dataframe
        self.img_dir = img_dir
        self.clip_model = clip_model
        self.preprocess_fn = preprocess_fn
        self.categories = dataframe['masterCategory'].unique().tolist()
        self.category_to_label = {category: i for i, category in enumerate(self.categories)}

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        img_id = self.dataframe.iloc[idx]['id']
        text = self.dataframe.iloc[idx]['productDisplayName']
        category = self.dataframe.iloc[idx]['masterCategory']

        img_path = os.path.join(self.img_dir, str(img_id) + '.jpg')
        image = Image.open(img_path)
        image_tensor = self.preprocess_fn(image)

        text_tokens = clip.tokenize([text])[0]

        label = self.category_to_label[category]

        return image_tensor, text_tokens, label

## Create data loaders

### Subtask:
Create PyTorch DataLoaders for the training and validation datasets to efficiently batch and load the data during training.


**Reasoning**:
Instantiate the FashionDataset for train and validation dataframes and create DataLoaders for them.



In [35]:
from torch.utils.data import DataLoader

train_dataset = FashionDataset(train_df, 'dataset/images', model, preprocess)
val_dataset = FashionDataset(val_df, 'dataset/images', model, preprocess)

batch_size = 32
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

## Define loss function and optimizer

### Subtask:
Choose an appropriate loss function (e.g., CrossEntropyLoss for classification) and an optimizer (e.g., Adam).


**Reasoning**:
Instantiate CrossEntropyLoss and the Adam optimizer.



In [36]:
import torch.nn as nn
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(classifier.parameters(), lr=0.001)

## Train the classifier

### Subtask:
Write the training loop to iterate over the dataloaders, perform forward and backward passes, and update the model weights. Include evaluation on the validation set during training to monitor performance.


**Reasoning**:
Implement the training loop including forward and backward passes, optimizer steps, and evaluation on the validation set, following steps 1-15 of the instructions.



In [37]:
num_epochs = 10

for epoch in range(num_epochs):
    # Training
    model.train()
    classifier.train()
    running_loss = 0.0
    for images, texts, labels in train_dataloader:
        images = images.to(device)
        texts = texts.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()

        with torch.no_grad():
            image_embeddings = model.encode_image(images)
            text_embeddings = model.encode_text(texts)

        concatenated_embeddings = torch.cat([image_embeddings, text_embeddings], dim=1)
        outputs = classifier(concatenated_embeddings)
        loss = criterion(outputs, labels)

        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)

    epoch_loss = running_loss / len(train_dataset)
    print(f'Epoch {epoch+1}/{num_epochs}, Training Loss: {epoch_loss:.4f}')

    # Validation
    model.eval()
    classifier.eval()
    correct = 0
    total = 0
    val_loss = 0.0
    with torch.no_grad():
        for images, texts, labels in val_dataloader:
            images = images.to(device)
            texts = texts.to(device)
            labels = labels.to(device)

            image_embeddings = model.encode_image(images)
            text_embeddings = model.encode_text(texts)

            concatenated_embeddings = torch.cat([image_embeddings, text_embeddings], dim=1)
            outputs = classifier(concatenated_embeddings)
            loss = criterion(outputs, labels)

            val_loss += loss.item() * images.size(0)

            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    epoch_val_loss = val_loss / len(val_dataset)
    accuracy = 100 * correct / total
    print(f'Epoch {epoch+1}/{num_epochs}, Validation Loss: {epoch_val_loss:.4f}, Validation Accuracy: {accuracy:.2f}%')


Epoch 1/10, Training Loss: 1.9617
Epoch 1/10, Validation Loss: 1.9625, Validation Accuracy: 8.33%
Epoch 2/10, Training Loss: 1.7605
Epoch 2/10, Validation Loss: 1.9375, Validation Accuracy: 8.33%
Epoch 3/10, Training Loss: 1.5894
Epoch 3/10, Validation Loss: 1.9071, Validation Accuracy: 16.67%
Epoch 4/10, Training Loss: 1.4368
Epoch 4/10, Validation Loss: 1.8957, Validation Accuracy: 16.67%
Epoch 5/10, Training Loss: 1.3044
Epoch 5/10, Validation Loss: 1.8902, Validation Accuracy: 16.67%
Epoch 6/10, Training Loss: 1.1835
Epoch 6/10, Validation Loss: 1.8904, Validation Accuracy: 16.67%
Epoch 7/10, Training Loss: 1.0723
Epoch 7/10, Validation Loss: 1.8974, Validation Accuracy: 16.67%
Epoch 8/10, Training Loss: 0.9726
Epoch 8/10, Validation Loss: 1.9059, Validation Accuracy: 16.67%
Epoch 9/10, Training Loss: 0.8813
Epoch 9/10, Validation Loss: 1.9223, Validation Accuracy: 16.67%
Epoch 10/10, Training Loss: 0.7991
Epoch 10/10, Validation Loss: 1.9427, Validation Accuracy: 16.67%


## Evaluate the trained classifier

### Subtask:
Evaluate the final performance of the trained classifier on the validation set.


**Reasoning**:
Evaluate the final performance of the trained classifier on the validation set by calculating the total validation loss and accuracy.



In [38]:
model.eval()
classifier.eval()

correct = 0
total = 0
val_loss = 0.0

with torch.no_grad():
    for images, texts, labels in val_dataloader:
        images = images.to(device)
        texts = texts.to(device)
        labels = labels.to(device)

        image_embeddings = model.encode_image(images)
        text_embeddings = model.encode_text(texts)

        concatenated_embeddings = torch.cat([image_embeddings, text_embeddings], dim=1)
        outputs = classifier(concatenated_embeddings)
        loss = criterion(outputs, labels)

        val_loss += loss.item() * images.size(0)

        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

epoch_val_loss = val_loss / len(val_dataset)
accuracy = 100 * correct / total

print(f'Final Validation Loss: {epoch_val_loss:.4f}, Final Validation Accuracy: {accuracy:.2f}%')

Final Validation Loss: 1.9427, Final Validation Accuracy: 16.67%


## Summary:

### Data Analysis Key Findings

*   The `FashionDataset` class was successfully created to handle the loading and preprocessing of image and text data using CLIP.
*   PyTorch `DataLoader` objects were successfully created for training and validation datasets with a batch size of 32, with the training data shuffled.
*   The `CrossEntropyLoss` function was chosen as the criterion for classification, and the Adam optimizer was selected with a learning rate of 0.001 to update the classifier's parameters.
*   The training process ran for 10 epochs, showing a consistent decrease in training loss.
*   Validation loss decreased initially but then increased, while validation accuracy plateaued at 16.67% after the third epoch.
*   The final evaluation on the validation set resulted in a loss of 1.9427 and an accuracy of 16.67%.

### Insights or Next Steps

*   The low validation accuracy suggests that the classifier is not performing well, likely due to overfitting to the small training dataset.
*   Next steps could include exploring data augmentation techniques, using a pre-trained classifier, or increasing the size of the training dataset to improve generalization.


In [45]:
from torchvision import transforms

# Define your augmentations for training set
train_transforms = transforms.Compose([
    transforms.Resize((224, 224)),   # Resize images to CLIP input size
    transforms.RandomHorizontalFlip(),  # Flip images horizontally 50% of time
    transforms.RandomRotation(15),       # Rotate images +/- 15 degrees
    transforms.ToTensor(),                # Convert to tensor
    transforms.Normalize((0.48145466, 0.4578275, 0.40821073),  # CLIP normalization means
                         (0.26862954, 0.26130258, 0.27577711))  # CLIP normalization stds
])

val_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize((0.48145466, 0.4578275, 0.40821073),
                         (0.26862954, 0.26130258, 0.27577711))
])


In [46]:
class FashionDataset(Dataset):
    def __init__(self, df, transform=None):
        self.df = df
        self.transform = transform
    ...
    def __getitem__(self, idx):
        img = Image.open(self.df.iloc[idx]['image_path']).convert('RGB')
        text = self.df.iloc[idx]['text']

        if self.transform:
            img = self.transform(img)

        # other code to return img, text, label remains unchanged


In [47]:
train_dataset = FashionDataset(train_df, transform=train_transforms)
val_dataset = FashionDataset(val_df, transform=val_transforms)


In [48]:
!pip install peft transformers --quiet


In [50]:
for name, _ in model.visual.named_modules():
    print(name)



conv1
ln_pre
transformer
transformer.resblocks
transformer.resblocks.0
transformer.resblocks.0.attn
transformer.resblocks.0.attn.out_proj
transformer.resblocks.0.ln_1
transformer.resblocks.0.mlp
transformer.resblocks.0.mlp.c_fc
transformer.resblocks.0.mlp.gelu
transformer.resblocks.0.mlp.c_proj
transformer.resblocks.0.ln_2
transformer.resblocks.1
transformer.resblocks.1.attn
transformer.resblocks.1.attn.out_proj
transformer.resblocks.1.ln_1
transformer.resblocks.1.mlp
transformer.resblocks.1.mlp.c_fc
transformer.resblocks.1.mlp.gelu
transformer.resblocks.1.mlp.c_proj
transformer.resblocks.1.ln_2
transformer.resblocks.2
transformer.resblocks.2.attn
transformer.resblocks.2.attn.out_proj
transformer.resblocks.2.ln_1
transformer.resblocks.2.mlp
transformer.resblocks.2.mlp.c_fc
transformer.resblocks.2.mlp.gelu
transformer.resblocks.2.mlp.c_proj
transformer.resblocks.2.ln_2
transformer.resblocks.3
transformer.resblocks.3.attn
transformer.resblocks.3.attn.out_proj
transformer.resblocks.3.ln_

In [52]:
for name, module in model.visual.named_modules():
    print(name)



conv1
ln_pre
transformer
transformer.resblocks
transformer.resblocks.0
transformer.resblocks.0.attn
transformer.resblocks.0.attn.out_proj
transformer.resblocks.0.ln_1
transformer.resblocks.0.mlp
transformer.resblocks.0.mlp.c_fc
transformer.resblocks.0.mlp.gelu
transformer.resblocks.0.mlp.c_proj
transformer.resblocks.0.ln_2
transformer.resblocks.1
transformer.resblocks.1.attn
transformer.resblocks.1.attn.out_proj
transformer.resblocks.1.ln_1
transformer.resblocks.1.mlp
transformer.resblocks.1.mlp.c_fc
transformer.resblocks.1.mlp.gelu
transformer.resblocks.1.mlp.c_proj
transformer.resblocks.1.ln_2
transformer.resblocks.2
transformer.resblocks.2.attn
transformer.resblocks.2.attn.out_proj
transformer.resblocks.2.ln_1
transformer.resblocks.2.mlp
transformer.resblocks.2.mlp.c_fc
transformer.resblocks.2.mlp.gelu
transformer.resblocks.2.mlp.c_proj
transformer.resblocks.2.ln_2
transformer.resblocks.3
transformer.resblocks.3.attn
transformer.resblocks.3.attn.out_proj
transformer.resblocks.3.ln_

In [53]:
# Unfreeze the CLIP visual encoder
for param in model.visual.parameters():
    param.requires_grad = True

# Use model.visual as is in your classifier without LoRA
clip_model_lora = model.visual


In [54]:
for param in model.visual.parameters():
    param.requires_grad = False

# Your classifier architecture remains the same


In [55]:
# Freeze CLIP visual encoder parameters
for param in model.visual.parameters():
    param.requires_grad = False

# Define your Multimodal classifier normally with model.visual as fixed encoder
# Training happens for only classifier parameters

# Here is an example of forward pass usage:
class MultimodalClassifier(nn.Module):
    def __init__(self, clip_visual, text_embed_dim, hidden_dim, num_classes):
        super().__init__()
        self.clip_visual = clip_visual
        self.text_embed_dim = text_embed_dim
        self.image_embed_dim = clip_visual.output_dim

        self.classifier_head = nn.Sequential(
            nn.Linear(self.image_embed_dim + text_embed_dim, hidden_dim),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(hidden_dim, num_classes)
        )

    def forward(self, image, text_emb):
        with torch.no_grad():
            img_emb = self.clip_visual(image)
        combined_emb = torch.cat([img_emb, text_emb], dim=1)
        return self.classifier_head(combined_emb)

# Initialize classifier
classifier = MultimodalClassifier(model.visual, 512, 512, len(category_list)).to(device)


In [56]:
# Freeze CLIP vision encoder to prevent training its weights
for param in model.visual.parameters():
    param.requires_grad = False


In [60]:
import torch.nn as nn

class MultimodalClassifier(nn.Module):
    def __init__(self, clip_visual, text_embed_dim, hidden_dim, num_classes):
        super().__init__()
        self.clip_visual = clip_visual
        self.text_embed_dim = text_embed_dim
        self.image_embed_dim = clip_visual.output_dim  # Make sure this attribute exists

        self.classifier_head = nn.Sequential(
            nn.Linear(self.image_embed_dim + text_embed_dim, hidden_dim),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(hidden_dim, num_classes)
        )

    def forward(self, image, text_emb):
        with torch.no_grad():
            img_emb = self.clip_visual(image)
        combined_emb = torch.cat([img_emb, text_emb], dim=1)
        return self.classifier_head(combined_emb)


hidden_dim = 512
text_embed_dim = 512
num_classes = len(category_list)  # category_list should be defined

classifier = MultimodalClassifier(model.visual, text_embed_dim, hidden_dim, num_classes).to(device)


In [61]:
import torch.nn as nn

class MultimodalClassifier(nn.Module):
    def __init__(self, clip_visual, text_embed_dim, hidden_dim, num_classes):
        super().__init__()
        self.clip_visual = clip_visual
        self.text_embed_dim = text_embed_dim
        self.image_embed_dim = clip_visual.output_dim  # Ensure this attribute exists

        self.classifier_head = nn.Sequential(
            nn.Linear(self.image_embed_dim + text_embed_dim, hidden_dim),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(hidden_dim, num_classes)
        )

    def forward(self, image, text_emb):
        with torch.no_grad():
            img_emb = self.clip_visual(image)
        combined_emb = torch.cat([img_emb, text_emb], dim=1)
        return self.classifier_head(combined_emb)


hidden_dim = 512
text_embed_dim = 512  # CLIP ViT-B/32 text embedding size
num_classes = len(category_list)  # Make sure category_list is defined earlier

classifier = MultimodalClassifier(model.visual, text_embed_dim, hidden_dim, num_classes).to(device)


In [63]:
for images, text_tokens, labels in train_dataloader:
    images = images.to(device)
    text_tokens = text_tokens.to(device)
    labels = labels.to(device)
    # Use in training loop next

In [65]:
import torch.optim as optim
import torch.nn.functional as F

optimizer = optim.Adam(classifier.parameters(), lr=1e-3)
epochs = 10  # You can increase later

for epoch in range(epochs):
    classifier.train()
    running_loss = 0

    for images, text_tokens, labels in train_dataloader:
        images, text_tokens, labels = images.to(device), text_tokens.to(device), labels.to(device)

        optimizer.zero_grad()

        # Encode text with frozen CLIP encoder
        with torch.no_grad():
            text_emb = model.encode_text(text_tokens)

        outputs = classifier(images, text_emb)
        loss = F.cross_entropy(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch {epoch+1}, Loss: {running_loss/len(train_dataloader):.4f}")

    # Optionally, add validation evaluation here

Epoch 1, Loss: 1.9254
Epoch 2, Loss: 1.3973
Epoch 3, Loss: 1.0621
Epoch 4, Loss: 0.7526
Epoch 5, Loss: 0.4853
Epoch 6, Loss: 0.3411
Epoch 7, Loss: 0.2251
Epoch 8, Loss: 0.1559
Epoch 9, Loss: 0.1015
Epoch 10, Loss: 0.0755


In [66]:
torch.save(classifier.state_dict(), 'multimodal_classifier_frozen_clip.pth')


In [67]:
classifier.load_state_dict(torch.load('multimodal_classifier_frozen_clip.pth'))
classifier.eval()


MultimodalClassifier(
  (clip_visual): VisionTransformer(
    (conv1): Conv2d(3, 768, kernel_size=(32, 32), stride=(32, 32), bias=False)
    (ln_pre): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    (transformer): Transformer(
      (resblocks): Sequential(
        (0): ResidualAttentionBlock(
          (attn): MultiheadAttention(
            (out_proj): NonDynamicallyQuantizableLinear(in_features=768, out_features=768, bias=True)
          )
          (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (mlp): Sequential(
            (c_fc): Linear(in_features=768, out_features=3072, bias=True)
            (gelu): QuickGELU()
            (c_proj): Linear(in_features=3072, out_features=768, bias=True)
          )
          (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        )
        (1): ResidualAttentionBlock(
          (attn): MultiheadAttention(
            (out_proj): NonDynamicallyQuantizableLinear(in_features=768, out_features=768, 

In [69]:
import torch
import torch.optim as optim
import torch.nn.functional as F

# Optimizer for classifier only
optimizer = optim.Adam(classifier.parameters(), lr=1e-3)

# Number of training epochs
epochs = 10

for epoch in range(epochs):
    classifier.train()
    running_loss = 0
    total_correct = 0
    total_samples = 0

    for images, text_tokens, labels in train_dataloader:
        images = images.to(device)
        text_tokens = text_tokens.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()

        # Encode text with frozen CLIP text encoder
        with torch.no_grad():
            text_emb = model.encode_text(text_tokens)

        # Forward pass through classifier
        outputs = classifier(images, text_emb)
        loss = F.cross_entropy(outputs, labels)

        loss.backward()
        optimizer.step()

        running_loss += loss.item()

        # Calculate accuracy on batch
        _, preds = torch.max(outputs, dim=1)
        total_correct += (preds == labels).sum().item()
        total_samples += labels.size(0)

    epoch_loss = running_loss / len(train_dataloader)
    epoch_acc = total_correct / total_samples
    print(f"Epoch {epoch+1}/{epochs} - Loss: {epoch_loss:.4f} - Accuracy: {epoch_acc*100:.2f}%")

Epoch 1/10 - Loss: 0.0554 - Accuracy: 100.00%
Epoch 2/10 - Loss: 0.0570 - Accuracy: 100.00%
Epoch 3/10 - Loss: 0.0231 - Accuracy: 100.00%
Epoch 4/10 - Loss: 0.0357 - Accuracy: 100.00%
Epoch 5/10 - Loss: 0.0128 - Accuracy: 100.00%
Epoch 6/10 - Loss: 0.0061 - Accuracy: 100.00%
Epoch 7/10 - Loss: 0.0088 - Accuracy: 100.00%
Epoch 8/10 - Loss: 0.0073 - Accuracy: 100.00%
Epoch 9/10 - Loss: 0.0044 - Accuracy: 100.00%
Epoch 10/10 - Loss: 0.0025 - Accuracy: 100.00%


In [70]:
def validate(model, classifier, val_loader):
    classifier.eval()
    total_correct = 0
    total_samples = 0
    running_loss = 0

    with torch.no_grad():
        for images, text_tokens, labels in val_loader:
            images = images.to(device)
            text_tokens = text_tokens.to(device)
            labels = labels.to(device)

            text_emb = model.encode_text(text_tokens)
            outputs = classifier(images, text_emb)
            loss = F.cross_entropy(outputs, labels)

            running_loss += loss.item()
            _, preds = torch.max(outputs, 1)
            total_correct += (preds == labels).sum().item()
            total_samples += labels.size(0)

    val_loss = running_loss / len(val_loader)
    val_acc = total_correct / total_samples

    print(f"Validation - Loss: {val_loss:.4f}, Accuracy: {val_acc*100:.2f}%")
    return val_loss, val_acc


In [71]:
torch.save(classifier.state_dict(), 'multimodal_classifier_frozen_clip.pth')


In [72]:
classifier.load_state_dict(torch.load('multimodal_classifier_frozen_clip.pth'))
classifier.eval()


MultimodalClassifier(
  (clip_visual): VisionTransformer(
    (conv1): Conv2d(3, 768, kernel_size=(32, 32), stride=(32, 32), bias=False)
    (ln_pre): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    (transformer): Transformer(
      (resblocks): Sequential(
        (0): ResidualAttentionBlock(
          (attn): MultiheadAttention(
            (out_proj): NonDynamicallyQuantizableLinear(in_features=768, out_features=768, bias=True)
          )
          (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (mlp): Sequential(
            (c_fc): Linear(in_features=768, out_features=3072, bias=True)
            (gelu): QuickGELU()
            (c_proj): Linear(in_features=3072, out_features=768, bias=True)
          )
          (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        )
        (1): ResidualAttentionBlock(
          (attn): MultiheadAttention(
            (out_proj): NonDynamicallyQuantizableLinear(in_features=768, out_features=768, 

In [73]:
# Load the saved model weights into your classifier
classifier.load_state_dict(torch.load('multimodal_classifier_frozen_clip.pth'))

# Put the model in evaluation mode for inference
classifier.eval()


MultimodalClassifier(
  (clip_visual): VisionTransformer(
    (conv1): Conv2d(3, 768, kernel_size=(32, 32), stride=(32, 32), bias=False)
    (ln_pre): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    (transformer): Transformer(
      (resblocks): Sequential(
        (0): ResidualAttentionBlock(
          (attn): MultiheadAttention(
            (out_proj): NonDynamicallyQuantizableLinear(in_features=768, out_features=768, bias=True)
          )
          (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (mlp): Sequential(
            (c_fc): Linear(in_features=768, out_features=3072, bias=True)
            (gelu): QuickGELU()
            (c_proj): Linear(in_features=3072, out_features=768, bias=True)
          )
          (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        )
        (1): ResidualAttentionBlock(
          (attn): MultiheadAttention(
            (out_proj): NonDynamicallyQuantizableLinear(in_features=768, out_features=768, 